# Elastic Search

___

## Elastic Search

config/jvm.options: 자바 heap 메모리 관련 설정 조작 가능(기본 1기가)

config/elasticsearch.yml: elastic search의 전반적 config.
- cluster.name
- node.name
- path.data: 데이터 저장 경로
- path.log: 로그 저장 경로
- network.host: 기본적으로 localhost, 환경 구축 완료된 ip로 입력하여 ip 변경 가능.
- http.port: 사용할 port 정보. 변경 가능.
- discovery.zen.unicast.hosts: 연결할 node들의 주소를 배열 내에 입력.

실행: bin/elasticsearch 입력 => 설정된 ip주소, port 정보대로 실행됨.

curl -XGET localhost:9200 입력 => 실행중인 elastic search 정보 출력됨.

tar xfz elasticsearch-6.3.2.tar.gz

mv elasticsearch-6.3.2 node-1

tar xfz elasticsearch-6.3.2.tar.gz

mv elasticsearch-6.3.2 node-2

tar xfz elasticsearch-6.3.2.tar.gz

mv elasticsearch-6.3.2 node-3

=> 3개의 node 폴더 생성.

각 node 폴더 => config/elasticsearch.yml 내용 변경 => 실행시 변경된 내용이 적용되어 elastic search 실행됨.

node-1 실행 중 bin/elasticsearch -E cluster.name=es-1 -E node.name=node-2 실행

=> 같은 cluster를 사용하여 node-1과 node-2 바인딩됨.

    curl -X PUT "localhost:9200/twitter/_doc/1" -H 'Content-Type: application/json' -d'
    {
        "user" : "kimchy",
        "post_date" : "2009-11-15T14:12:12",
        "message" : "trying out Elasticsearch"
    }
    '

입력 => 9200번 port에 twitter라는 index가 자동으로 생성되어 내용 입력됨.

curl localhost:9200/twitter/_doc/1

curl localhost:9201/twitter/_doc/1

둘 다 같은 결과 반환: 두 node들이 하나의 cluster에 바인딩되어있기 때문.

bin/elasticsearch -E cluster.name=es-2 -E node.name=node-3 실행 => node-1, node-2와 다른 es-2라는 cluster로 바인딩되어 실행됨.

    curl -XPUT localhost:9202/test/_doc/1 -d'
     { "message": "happy" }' -H 'Content-Type: application/json'
     
localhost:9200, localhost:9201에서는 내용 못가져옴, localhost:9202에서만 가져올 수 있음.

#### 결론: 같은 머신에서라도 cluster가 다르면 별도로 저장된다.

___

## Kibana

node 켜놓은 상태로 kibana 설치 폴더에서 bin/kibana: kibana 실행됨.

기본 port: 5601번 port 사용.

Dev Tools: Elastic Search 작업을 편리하게 할 수 있는 환경 제공.
- 명령어 입력 후 command + Enter => 결과 화면 출력됨.
- 명령어를 CRUD로 copy하여 shell에서 작업하는 것도 가능.
- GET: 검색시 사용.
    * _search: 검색 api.
        - index/_search: 정해진 index 내에서만 검색한다.
        - GET _search: 전체 index 검색.
            * 옵션 주지 않으면 기본적으로 전체 document 검색.
    * index당 하나의 type만 허용된다.
    * _mapping: field 정보를 가져온다.
        - keyword field: 검색어로 사용하기 위해 쪼개는 작업에서 제외되는 내용.
- DELETE: 삭제시 사용.
    * DELETE index: index 전체 삭제.
    * DELETE index/type/doc_name: document만 삭제.
- PUT: index 생성.
    * "settings" 내용 변경하여 설정 변경 가능.
- POST: document 색인시 사용.
    * 다량의 document에 대해 작업할 시에는 반드시 bulk API 사용.
        - 예: POST libarary/books/_bulk ...

### Search

    GET library/_search
    {
      "query": {
        "match": {
          "title": "fox"
        }
      }
    }
title에 fox가 들어간 모든 document 검색됨.

    GET library/_search
    {
      "query": {
        "match": {
          "title": "quick dog"
        }
      }
    }
띄어쓰기로 구분 => or 조건으로 띄어쓰기로 구분된 모든 단어에 대해 검색 실행.
- score 달라짐: 명시된 단어가 모두 들어있고, 많이 출현할수록 높은 score.
- score가 높은 순으로 자동 정렬됨.


    GET library/_search
    {
      "query": {
        "match_phrase": {
          "title": "quick dog"
        }
      }
    }
quick dog 전체가 들어있는 문서를 찾는다.

relevance(정확도): score 점수를 계산하는 알고리즘.
- TF-IDF 알고리즘을 주로 사용한다.
    * TF: 검색 field 내에서 주어진 단어가 몇번이나 나왔는지 확인.
    * IDF: 전체 문서 내에서 주어진 단어가 몇번이나 나왔는지 확인.

    GET library/_search
    {
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "title": "quick"
              }
            },
            {
              "match_phrase": {
                "title": {
                  "query": "lazy dog"
                }
              }
            }
          ]
        }
      }
    }
must 옵션: 주어진 조건에 대해 and 조건으로 검색 실행 => title field에 quick이 등장하고 lazy dog이라는 문구가 등장하는 문서를 찾는다.

    GET library/_search
    {
      "query": {
        "bool": {
          "must_not": [
            {
              "match": {
                "title": "lazy"
              }
            },
            {
              "match_phrase": {
                "title": {
                  "query": "quick dog"
                }
              }
            }
          ]
        }
      }
    }
must_not 옵션: 주어진 조건을 모두 만족하지 않는 문서 검색.

    GET library/_search
    {
      "query": {
        "bool": {
          "should": [
            {
              "match_phrase": {
                "title": "quick dog"
              }
            },
            {
              "match_phrase": {
                "title": {
                  "query": "lazy dog",
                  "boost": 3
                }
              }
            }
          ]
        }
      }
    }
should 옵션: quick dog, lazy dog가 없어도 검색이 되나 lazy dog가 포함된 경우 더 높은 score를 부여한다.(or과 유사하나 score 부여하는 것이 다름)
- boost 값 조절 가능.(기본값 1)
- 여러 단어 검색시 특정 단어에 대해 더 많은 비중을 두고자 할때 사용.


    GET library/_search
    {
      "query": {
        "bool": {
          "should": [
            {
              "match": {
                "title": "lazy"
              }
            }
          ],
          "must": [
            {
              "match":{
                "title": "dog"
              }
            }
          ]
        }
      }
    }
must, should 혼합: must에 명시된 조건은 반드시 들어가야 하고 should 조건에 matching될 경우 score boost.

    GET library/_search
    {
      "query": {
        "bool": {
          "should": [
            {
              "match_phrase": {
                "title": {
                  "query": "quick dog",
                  "boost": 2
                }
              }
            },
            {
              "match_phrase": {
                "title": {
                  "query": "lazy dog"
                }
              }
            }
          ]
        }
      },
      "highlight": {
        "fields": {
          "title": {}
        }
      }
    }
highlight 옵션: 검색어와 매칭된 부분을 highlight.
- 검색 결과값이 크거나 여러개의 field를 사용하는 경우 유용함.
- 어느 부분에서 어떤 field가 검색되었는지 한눈에 알 수 있다.


    GET library/_search
    {
      "query": {
        "bool": {
          "must": [
            {
              "match": {
                "title": "dog"
              }
            }
          ],
          "filter": {
            "range": {
              "price": {
                "gte": 5,
                "lte": 10
              }
            }
          }
        }
      }
    }
filter 옵션: 검색 결과에 대한 subset을 가져오는 개념.
- score 계산하지 않고 캐싱 => query보다 속도가 빠르다.
- 사용 조건 제한: bool 조건일 경우에만.
- range 조건: price가 5 이상, 10 이하인 결과를 찾는다.
- filter 조건에 맞는다고 score가 더해지는건 아님: score를 계산하지 않기 때문 => query보다 속도가 빠르다.
- score가 필요없는 경우 유용: 숫자값 검색시 유용함.


    GET library/_search
    {
      "query": {
        "bool": {
          "filter": {
            "range": {
              "price": {
                "gt": 5
              }
            }
          }
        }
      }
    }
price 값이 5 이상인 document만 가져온다.

___

### Analysis
- text field가 elastic search에 들어갈 때 text가 가공되어 분석에 사용할 수 있는 값으로 바뀌어 들어간다.
- term: 검색에 사용 가능한 검색어.


    GET library/_analyze
    {
      "tokenizer": "standard",
      "text": "Brown fox brown dog"
    }
- tokenizer: 데이터를 쪼개는 역할.
- text: 쪼개질 데이터.
- 결과: 어떻게 쪼개지는지 보여줌.


    GET library/_analyze
    {
      "tokenizer": "standard",
      "filter":[
        "lowercase"
      ],
      "text": "Brown fox brown dog"
    }
대문자, 소문자를 가리지 않고 검색할 수 있도록 설정.

    GET library/_analyze
    {
      "tokenizer": "standard",
      "filter":[
        "lowercase",
        "unique"
      ],
      "text": "Brown brown brown fox brown dog"
    }
중복되는 term은 제거하도록 설정.

    GET library/_analyze
    {
      "analyzer": "standard", 
      "text": "Brown fox brown dog"
    }
- analyzer: standard => standard tokenizer를 사용하도록 설정(elastic search가 기본적으로 제공하는 tokenizer 사용)
- tokenizer를 index로 저장한 후 불러와 사용자 정의의 tokenizer를 사용하는 것도 가능.

### Complex sentence analysis
    GET library/_analyze
    {
      "tokenizer": "standard",
      "filter": [
        "lowercase"
      ],
      "text": "THE quick.brown_FOx jumped! $19.95 @ 3.0"
    }
- 결과: the, quick.brown_fox, jumped, 19.95, 3.0
- 특수문자 사라짐, 철자 사이의 특수문자는 계속 유지됨, 숫자 유지됨.


    GET library/_analyze
    {
      "tokenizer": "letter",
      "filter": [
        "lowercase"
      ],
      "text": "THE quick.brown_FOx jumped! $19.95 @ 3.0"
    }
- 결과: the, quick, brown, fox, jumped
- 의미가 있는 영문자만 남김, 숫자 사라짐.


    GET library/_analyze
    {
      "tokenizer": "standard",
      "text": "elastic@example.com website: https://www.elastic.co"
    }
결과: elastic, example.com, website, https, www.elastic.co

    GET library/_analyze
    {
      "tokenizer": "uax_url_email",
      "text": "elastic@example.com website: https://www.elastic.co"
    }
- 결과: elastic@example.com, website, https://www.elastic.co
- email 주소, url 보존됨. email 주소일 경우 EMAIL, url일 경우 URL로 타입 자동 지정.

### Aggregation
- 기본적으로 doc value값 사용.
- 일반적으로 keyword type의 값들만 사용 가능.
- query와 동시 사용 가능.


    GET library/_search
    {
      "size": 0,
      "aggs": {
        "popular-colors": {
          "terms": {
            "field": "colors.keyword"
          }
        }
      }
    }
library의 color들이 각각 몇개의 document에서 등장했는지 집계.

    GET library/_search
    {
      "query": {
        "match": {
          "title": "dog"
        }
      },
      "aggs": {
        "popular-colors": {
          "terms": {
            "field": "colors.keyword"
          }
        }
      }
    }
우선 query에 매칭되는 검색 결과를 불러오고 각각의 color들이 몇개의 document에서 등장했는지 집계.

    GET library/_search
    {
      "size": 0,
      "aggs": {
        "price-statistics": {
          "stats": {
            "field": "price"
          }
        },
        "popular-colors": {
          "terms": {
            "field": "colors.keyword"
          },
          "aggs": {
            "avg-price-per-color": {
              "avg": {
                "field": "price"
              }
            }
          }
        }
      }
    }
- 결과: price에 대한 통계 정보(count, min, max, avg, sum), 각 color가 몇개의 document에서 등장했는지, color에 대한 price 평균.
- 여러 aggregation을 동시에 실행할 수 있다.

### Updating document
    GET library/books/1
단일 document 정보를 불러온다.

    POST library/books/1
    {
      "title": "The quick brow fox",
      "price": 10,
      "colors": ["red", "green", "blue"]
    }
- document의 내용이 POST 메소드에서 입력한 내용으로 대체됨.
- version 올라감.


    POST library/books/1/_update
    {
      "doc": {
        "title": "The quick fantastic fox"
      }
    }
- _update api 사용: 명시된 field만 변경 가능.
- 내부적으로는 전체 document를 가져온 다음 명시된 field만 바꿔준 다음 원본 document를 다시 입력하는 방식으로 동작 => 완전한 부분 update는 elastic search에서 존재하지 않는다.

### Mapping
    GET library/_mapping
index의 계층 구조를 보여준다.

    PUT famous-librarians
    {
      "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 0,
        "analysis": {
          "analyzer": {
            "my-desc-analyzer": {
              "type": "custom",
              "tokenizer": "uax_url_email",
              "filter": [
                "lowercase"
              ]
            }
          }
        }
      },
      "mappings": {
        "librarian": {
          "properties": {
            "name": {
              "type": "text"
            },
            "favorite-colors": {
              "type": "keyword"
            },
            "birth-date": {
              "type": "date",
              "format": "year_month_day"
            },
            "hometown": {
              "type": "geo_point"
            },
            "description": {
              "type": "text",
              "analyzer": "my-desc-analyzer"
            }
          }
        }
      }
    }
- PUT: 새로운 index 생성.
- settings: index가 가지는 system적 설정.
- analyzer: custom analyzer 생성.
- mappings: 스키마와 유사. 각 속성의 type, 사용할 analyzer 설정 => 설정된 값대로 필드 값 색인됨.
    * mappings에 정의되지 않은 새로운 field 정보가 들어올 경우: mappings가 자동으로 업데이트됨.
    * 미리 만들어놓은 mappings 변경 불가능 => 변경하려면 index 지우고 새로 색인해야함.
    
    
    PUT famous-librarians/librarian/1
    {
      "name": "Sarah Byrd Askew",
      "favorite-colors": ["Yellow", "light-grey"],
      "birth-date": "1877-02-15",
      "hometown": {
        "lat": 32.349722,
        "lon": -87.641111
      },
      "description":"An American public librarian who pioneered the establishment of country libraries in the United States - https://en.wikipedia.org/wiki/Sarah_Byrd_Askew"
    }

    PUT famous-librarians/librarian/2
    {
      "name": "John J. Beckley",
      "favorite-colors": ["Red", "off-white"],
      "birth-date": "1757-08-07",
      "hometown": {
        "lat": 51.507222,
        "lon": -0.1275
      },
      "description":"An American political campaign manager and the first librarian of the United States Congress - https://en.wikipedia.org/wiki/John_J._Beckley"
    }
데이터 삽입

    GET famous-librarians/_search
    {
      "query": {
        "query_string": {
          "fields": [
            "favorite-colors"
          ],
          "query": "yellow OR off-white"
        }
      }
    }
결과: 2번 document만 검색됨: favorite-colors에 대해 term을 쪼개거나 lowercase로 만드는 과정을 거치지 않았기 때문에 yellow에 대해서는 검색되지 않는다.(Yellow로 바꾸면 2개 다 나옴)

    GET famous-librarians/_search
    {
      "query": {
        "bool": {
          "must": [
            {
              "match_all": {}
            }
          ],
          "filter": {
            "range": {
              "birth-date": {
                "gte": "now-200y",
                "lte": "2000-01-01"
              }
            }
          }
        }
      }
    }
- 날짜에 대해 범위 설정하여 검색 가능
- now: 현재 날짜.
- 현재부터 200년 전 ~ 2000년 1월 1일 사이의 birth-date 검색.


    GET famous-librarians/_search
    {
      "query": {
        "bool": {
          "filter": {
            "geo_distance": {
              "distance": "100km",
              "hometown": {
                "lat": 32.41,
                "lon": -86.92
              }
            }
          }
        }
      }
    }
- 특정 지점에서 100km 이내의 위치 검색 가능.
- 응용: 특정 지점에서 가까운 호텔 검색.